In [ ]:
import tensorflow as tf
from keras.datasets import mnist
from keras.models import *
from keras.layers import *

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

In [ ]:
class MyModel(Sequential):

	def __init__(self, **kwargs):

		super().__init__(**kwargs)


	def compile(self, **kwargs):

		super().compile(**kwargs)

		self.acculumated_gradients = [
			tf.Variable(
				initial_value = tf.zeros_like(v, dtype = tf.float32),
				trainable = False,
				dtype = tf.float32
			)
			for v in self.trainable_variables
		]

		self.acculation_counter = tf.Variable(
			initial_value = 0,
			trainable = False,
			dtype = tf.int32
		)

	@tf.function
	def train_step(self, data):

		x, y = data

		with tf.GradientTape() as tape:
			pred = self(x, training = True)
			loss = self.compiled_loss(y, pred, regularization_losses = self.losses)

		grads = tape.gradient(loss, self.trainable_variables)

		for i in range(len(grads)):
			self.acculumated_gradients[i].assign_add(grads[i] / 16.0)

		if self.acculation_counter >= 16:

			self.optimizer.apply_gradients(zip(self.acculumated_gradients, self.trainable_variables))

			for i in range(len(grads)):
				self.acculumated_gradients[i].assign(tf.zeros_like(grads[i], dtype = tf.float32))

			self.acculation_counter.assign(0)

		self.acculation_counter.assign_add(1)

		#self.optimizer.apply_gradients(zip(grads, self.trainable_variables))

		self.compiled_metrics.update_state(y, pred)

		return {m.name: m.result() for m in self.metrics}

In [ ]:
model = MyModel()

model.add(Conv2D(32, (3, 3), activation = 'relu', input_shape = (28, 28, 1)))
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D())

model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dense(10, activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs = 3, batch_size = 4, validation_data = (x_test, y_test))

In [ ]:
print(list(history.history['loss']))
print(list(history.history['val_loss']))

In [ ]:
import numpy as np

a = np.random.normal(0.5, 2, 100)

# apply softmax
b = np.exp(a) / np.sum(np.exp(a))

probabilities = b
print(list(probabilities))
probabilities = np.log(probabilities) / 0.5
print(list(probabilities))
probabilities = np.exp(probabilities) / np.sum(np.exp(probabilities))
print(list(probabilities))

sorted_indices = np.argsort(probabilities)[::-1]
print(list(sorted_indices))
cumulative_probabilities = np.cumsum(probabilities[sorted_indices])
print(list(cumulative_probabilities))
sorted_indices = sorted_indices[cumulative_probabilities <= 1.0]
print(list(sorted_indices))
probabilities = probabilities[sorted_indices]
print(list(probabilities))
probabilities = probabilities / np.sum(probabilities)
print(list(probabilities))